## pandasでリストとして格納されている各要素をカラムとして設定し、ワンホットエンコードとして展開する


データ分析をしていて、pandasの要素にリストが格納されており、そのリストに対してワンホットエンコードした状態のDataFrameを作りたいという機会があり、結構苦労したのでメモしておきます。

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/article/library/pandas/02/pandas_02_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/article/library/pandas/02/pandas_02_nb.ipynb)

### 実行環境
筆者のOSはmacOSです。LinuxやUnixのコマンドとはオプションが異なります。

In [31]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [32]:
!python -V

Python 3.9.17


基本的なライブラリをインポートしそのバージョンを確認しておきます。

In [33]:
%matplotlib inline

import pandas as pd

print('pandas version :', pd.__version__)

pandas version : 2.0.3


### サンプルデータの準備

In [34]:
df = pd.DataFrame(
    {
        "user_id": ["A", "B", "C"],
        "item_id": [["PC", "Book", "Water"], ["Book", "Table"], ["Desk", "CD"]],
    }
)

df.head()

,user_id,item_id
0,A,"[PC, Book, Water]"
1,B,"[Book, Table]"
2,C,"[Desk, CD]"


### MultiLabelBinarizerの利用

結論から述べると、`MultiLabelBinarizer` というscikit-learnのライブラリを利用します。

以下のように、 `fit_transform`を利用する事で、ワンホットエンコードを簡単に実現できます。また、それに対応するカラム名も簡単に取得できます。

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
mlb.fit_transform(df.item_id)

array([[1, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0]])

In [36]:
mlb.classes_

array(['Book', 'CD', 'Desk', 'PC', 'Table', 'Water'], dtype=object)

あとはこれを組み合わせるだけです。popでdfから取りだして、最後にjoinで結合します。

In [37]:
out_df = df.join(pd.DataFrame(mlb.fit_transform(df.pop("item_id")), columns=mlb.classes_))

out_df

,user_id,Book,CD,Desk,PC,Table,Water
0,A,1,0,0,1,0,1
1,B,1,0,0,0,1,0
2,C,0,1,1,0,0,0


## 参考サイト

- https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list